In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import json

inicio = time.time()

base_url = 'https://www.idealista.com'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0',
    'Accept-Language': 'es-ES,es;q=0.9',
    'Referer': 'https://www.google.com/'
}

urls_y_situaciones = {
    'https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-nuda-propiedad/': 'Nuda propiedad',
    'https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-inquilino/': 'Alquilada, con inquilinos',
    'https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-ocupado/': 'Ocupada ilegalmente',
    'https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-sin-inquilinos/': 'Disponible, sin ninguna de las restricciones anteriores'
}

resultados = []

with open('idealista_resultados1.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([
        'Título', 'Precio', 'Tipo de vivienda', 'Zona', 'Habitaciones', 'Metros cuadrados',
        'Baños', 'Planta', 'Estado', 'Situación vivienda', 'Tipo de anuncio', 'Multimedia',
        'Aire acondicionado', 'Armarios empotrados', 'Ascensor', 'Balcón o terraza',
        'Exterior', 'Garaje', 'Jardín', 'Piscina', 'Trastero', 'Vivienda accesible',
        'Vivienda de lujo', 'Descripción',
        'Metros cabecera',
        'Orientación', 'Construido en', 'Acceso movilidad', 'Calefacción',
        'Consumo energético', 'Emisiones', 'Equipamiento extra'
    ])

    for base_list_url, situacion_fijada in urls_y_situaciones.items():
        print(f"\nScrapeando: {situacion_fijada} - {base_list_url}")
        pagina = 1
        url_anuncio_anterior = None
        while True:
            if pagina == 1:
                list_url = base_list_url 
            else:
                list_url = base_list_url.rstrip('/') + f'/pagina-{pagina}.htm'
            print(f"\nScrapeando: {situacion_fijada} - {list_url}")
            print(f"  -> Página {pagina}...", end=' ')
            response = requests.get(list_url, headers=headers)

            if response.status_code != 200:
                print("no existe")
                break

            soup = BeautifulSoup(response.content, 'html.parser')
            listings = soup.find_all('article')
            if not listings:
                print("sin anuncios")
                break
            first_listing = listings[0].find('a', class_='item-link')
            url_anuncio_actual = first_listing['href'] if first_listing and first_listing.has_attr('href') else None
        
            if pagina == 1:
                url_anuncio_primera_pagina = url_anuncio_actual#solo se compara con el primer anuncio de la primera pagian, proque cuando la pagina no existe redirige a la primera
            else:
                if url_anuncio_actual == url_anuncio_primera_pagina:
                    print("repetido (redirigido a página 1)")
                    break
            print(f"{len(listings)} anuncios")

            for listing in listings:
                title_tag = listing.find('a', class_='item-link')
                title = title_tag.get_text(strip=True) if title_tag else 'N/A'
                price_tag = listing.find('span', class_='item-price')
                price = price_tag.get_text(strip=True) if price_tag else 'N/A'
                link = base_url + title_tag['href'] if title_tag and title_tag.has_attr('href') else None

                # Inicialización de campos
                tipo_vivienda = situacion = baños = estado = planta_anuncio = descripcion = 'N/A'
                habitaciones_anuncio = metros_anuncio = situacion_vivienda = 'N/A'
                caracteristicas = []
                multimedia = "Sin fotos"
                tipo_anuncio = fecha_publicacion = 'N/A'
                metros_construidos = habitaciones_cabecera = planta_cabecera = 'N/A'
                metros_utiles = orientacion = construido_en = acceso_movilidad = calefaccion = 'N/A'
                consumo_energetico = emisiones_energeticas = 'N/A'
                equipamiento_extra = []
                situacion_vivienda = situacion_fijada

                aire_acondicionado = armarios = ascensor = balcon = exterior = garaje = jardin = piscina = trastero = accesible = lujo = 'No'

                if link:
                    anuncio_res = requests.get(link, headers=headers)
                    if anuncio_res.status_code == 200:
                        anuncio_soup = BeautifulSoup(anuncio_res.content, 'html.parser')

                        title_tag = anuncio_soup.find('title')
                        if title_tag:
                            title_text = title_tag.get_text(strip=True).lower()
                            if "piso" in title_text:
                                tipo_vivienda = "Piso"
                            elif "ático" in title_text:
                                tipo_vivienda = "Ático"
                            elif "dúplex" in title_text or "duplex" in title_text:
                                tipo_vivienda = "Dúplex"
                            elif "independiente" in title_text:
                                tipo_vivienda = "Casa o chalet independiente"
                            elif "pareado" in title_text:
                                tipo_vivienda = "Chalet pareado"
                            elif "adosado" in title_text:
                                tipo_vivienda = "Chalet adosado"
                            elif "rústica" in title_text or "cortijo" in title_text or "rural" in title_text:
                                tipo_vivienda = "Casa rústica"
                            elif "estudio" in title_text:
                                tipo_vivienda = "Estudio"

                        location_tag = anuncio_soup.find('span', class_='main-info__title-minor')
                        situacion = location_tag.get_text(strip=True) if location_tag else 'N/A'

                        cabecera = anuncio_soup.find_all('div', class_='info-features')
                        if cabecera:
                            cab_texts = cabecera[0].get_text(" | ", strip=True).split(" | ")
                            for text in cab_texts:
                                if "m²" in text:
                                    metros_anuncio = text
                                elif "hab" in text:
                                    habitaciones_anuncio = text
                                elif "Planta" in text:
                                    planta_anuncio = text

                        desc_tag = anuncio_soup.find('div', {'class': 'adCommentsLanguage'})
                        descripcion = desc_tag.get_text(strip=True) if desc_tag else 'Sin descripción'

                        caracteristicas_raw = anuncio_soup.find_all('li')
                        for li in caracteristicas_raw:
                            texto = li.get_text(strip=True)
                            caracteristicas.append(texto)

                            if "m² construidos" in texto:
                                metros_construidos = texto
                            elif "baño" in texto:
                                baños = texto
                            elif "Planta" in texto:
                                planta = texto
                            elif "segunda mano" in texto.lower() or "nuevo" in texto.lower():
                                estado = texto
                            elif "orientación" in texto.lower():
                                orientacion = texto
                            elif "construido en" in texto.lower():
                                construido_en = texto
                            elif "movilidad reducida" in texto.lower():
                                acceso_movilidad = texto
                            elif "calefacción" in texto.lower():
                                calefaccion = texto
                            elif "consumo:" in texto.lower():
                                consumo_energetico = texto
                            elif "emisiones:" in texto.lower():
                                emisiones_energeticas = texto
                            elif "aire acondicionado" in texto.lower():
                                aire_acondicionado = 'Sí'
                            elif "armarios empotrados" in texto.lower():
                                armarios = 'Sí'
                            elif "ascensor" in texto.lower():
                                ascensor = 'Sí'
                            elif "balcón" in texto or "terraza" in texto.lower():
                                balcon = 'Sí'
                            elif "exterior" in texto.lower():
                                exterior = 'Sí'
                            elif "garaje" in texto.lower():
                                garaje = 'Sí'
                            elif "jardín" in texto.lower():
                                jardin = 'Sí'
                            elif "piscina" in texto.lower():
                                piscina = 'Sí'
                            elif "trastero" in texto.lower():
                                trastero = 'Sí'
                            elif "accesible" in texto.lower():
                                accesible = 'Sí'
                            elif "lujo" in texto.lower():
                                lujo = 'Sí'

                        for div in anuncio_soup.find_all('div'):
                            heading = div.find(['h2', 'h3'])
                            if heading and 'equipamiento' in heading.get_text(strip=True).lower():
                                ul = div.find('ul')
                                if ul:
                                    for li in ul.find_all('li'):
                                        texto = li.get_text(strip=True)
                                        if texto and texto not in equipamiento_extra:
                                            equipamiento_extra.append(texto)
                                break

                        imagenes = anuncio_soup.find_all('img', src=True)
                        fotos_reales = [
                            img['src'] for img in imagenes
                            if any(palabra in img['src'] for palabra in ['image.master', 'img4.idealista.com', 'foto'])
                        ]
                        multimedia = "Con fotos" if fotos_reales else "Sin fotos"

                        tipo_tag = anuncio_soup.find('div', class_='professional-name')
                        tipo_anuncio = 'Profesional' if tipo_tag else 'Particular'

                    time.sleep(1)

                writer.writerow([
                    title, price, tipo_vivienda, situacion, habitaciones_anuncio, metros_anuncio,
                    baños, planta_anuncio, estado, situacion_vivienda, tipo_anuncio,
                    multimedia, aire_acondicionado, armarios, ascensor, balcon, exterior,
                    garaje, jardin, piscina, trastero, accesible, lujo, descripcion,
                    metros_construidos, orientacion, construido_en, acceso_movilidad,
                    calefaccion, consumo_energetico, emisiones_energeticas, ", ".join(equipamiento_extra)
                ])

                resultados.append({
                    "Título": title, "Precio": price, "Tipo de vivienda": tipo_vivienda, "Zona": situacion,
                    "Habitaciones": habitaciones_anuncio, "Metros cuadrados": metros_anuncio, "Baños": baños, "Planta": planta_anuncio,
                    "Estado": estado, "Situación vivienda": situacion_vivienda, "Tipo de anuncio": tipo_anuncio, "Multimedia": multimedia,
                    "Aire acondicionado": aire_acondicionado, "Armarios empotrados": armarios,
                    "Ascensor": ascensor, "Balcón o terraza": balcon, "Exterior": exterior,
                    "Garaje": garaje, "Jardín": jardin, "Piscina": piscina, "Trastero": trastero,
                    "Vivienda accesible": accesible, "Vivienda de lujo": lujo, "Descripción": descripcion,
                    "Metros construidos/utiles": metros_construidos,
                    "Orientación": orientacion, "Construido en": construido_en,
                    "Acceso movilidad": acceso_movilidad, "Calefacción": calefaccion,
                    "Consumo energético": consumo_energetico, "Emisiones": emisiones_energeticas,
                    "Equipamiento extra": equipamiento_extra
                })

            pagina += 1

with open('idealista_resultados1.json', 'w', encoding='utf-8') as json_file:
    json.dump(resultados, json_file, ensure_ascii=False, indent=4)

print("\n Datos guardados en idealista_resultados1.csv y idealista_resultados1.json")

fin = time.time()
duracion = fin - inicio

print(f"\n Tiempo total de ejecución: {duracion:.2f} segundos")